# Merge CIRI data with scraped State Dep Reports
## Merge on year and country name

In [2]:
import pandas as pd


In [3]:
ciri_df = pd.read_csv('../data/CIRI_Data_1981_2011.csv')

In [7]:
ciri_df.head()

,CTRY,YEAR,CIRI,COW,POLITY,UNCTRY,UNREG,UNSUBREG,PHYSINT,DISAP,...,OLD_MOVE,SPEECH,ELECSD,OLD_RELFRE,NEW_RELFRE,WORKER,WECON,WOPOL,WOSOC,INJUD
0,Afghanistan,1981,101,700.0,700.0,4.0,142,62,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,1982,101,700.0,700.0,4.0,142,62,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,Afghanistan,1983,101,700.0,700.0,4.0,142,62,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Afghanistan,1984,101,700.0,700.0,4.0,142,62,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Afghanistan,1985,101,700.0,700.0,4.0,142,62,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
state_dep = pd.read_csv('../data/Custom_State_Dep_Reports/All_report_1999_2009.csv')

In [6]:
state_dep.head()

,Unnamed: 0,Country,Text,Year
0,0,Angola,The Republic of Angola's transition from a sin...,1999
1,1,Benin,The Republic of Benin is a constitutional demo...,1999
2,2,Botswana,"Botswana is a longstanding, multiparty democra...",1999
3,3,Burkina Faso,President Blaise Compaore continued to dominat...,1999
4,4,Burundi,Burundi is ruled by an authoritarian military ...,1999


In [9]:
df_text_to_ciri = pd.merge(state_dep,ciri_df,left_on=['Country','Year'],right_on=['CTRY','YEAR'], how='inner')

In [14]:
df_text_to_ciri = df_text_to_ciri.drop(columns=['Unnamed: 0','Country'])

In [15]:
df_text_to_ciri.to_csv('../data/Custom_State_Dep_Reports/CIRI_Text_1999_2009.csv')

In [21]:
len(df_text_to_ciri['CTRY'].unique())

192

In [22]:
len(state_dep['Country'].unique())


246